In [ ]:
import requests
import re

In [ ]:
def get_domain_rules(domain):
    res = requests.get(f'{domain}/robots.txt')
    if res.status_code == 200:
        return res.text
    return None

rules = get_domain_rules('https://tesla.com')
rules

In [ ]:
def filter_disallow_rules(rules):
    rulelist = rules.splitlines()
    disallows = []
    for rule in rulelist:
        if rule.startswith('Disallow: '):
            disallows.append(rule.split(': ')[1])
    return disallows

In [ ]:
disallows = filter_disallow_rules(rules)
disallows

In [ ]:
def filter_paths(paths, filters):
    res = paths[:]
    for f in filters:
        res = list(filter(lambda p: not re.search(f, p), res))
    return res

In [ ]:
filters = [
    r'[*]', # * is used for generalization, not a unique file
]
filter_paths(disallows, filters)